## Build Environment

In [ ]:
# "man", "woman", or "person".
dataset = "man"
# A single-word token.
token = project_name = "project_name"

max_training_steps = 1000
hf_token ="HUGGINGFACE_TOKEN"

In [ ]:
if dataset == "man":
    dataset="man_euler"
    class_word = "man"
elif dataset == "woman":
    dataset="woman_ddim"
    class_word = "woman"
elif dataset == "person":
    dataset="person_ddim"
    class_word = "person"
else:
    raise ValueError("Invalid dataset.")

!mkdir training_samples

In [ ]:
#BUILD ENV
!pip install omegaconf jupyternotify
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1

In [4]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [ ]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token(hf_token)

In [6]:
## Download the 1.4 sd model
import os
if not os.path.exists("model.ckpt"):
    from huggingface_hub import hf_hub_download
    downloaded_model_path = hf_hub_download(
     repo_id="CompVis/stable-diffusion-v-1-4-original",
     filename="sd-v1-4.ckpt",
     use_auth_token=True
    )

    ## Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
    actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
    !mv {actual_locations_of_model_blob[-1]} model.ckpt

OSError: You specified use_auth_token=True, but a huggingface token was not found.

## Download pre-generated regularization images

We've created the following image sets

* man_euler - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* man_unsplash - pictures from various photographers
* person_ddim
* woman_ddim - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended

In [ ]:
# Grab the existing regularization images
# Choose the dataset that best represents what you are trying to do and matches what you used for your token
# man_euler, man_unsplash, person_ddim, woman_ddim
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p outputs/txt2img-samples/samples/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* outputs/txt2img-samples/samples/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_samples
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body 
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make (most of the time, that means no selfies).
- 

In [ ]:
!sed -i s/joepenna/{token}/g ldm/data/personalized.py

In [ ]:
reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples/" + dataset

!rm -rf training_samples/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root {reg_data_root} \
 -n {project_name} \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_samples" \
 --max_training_steps {max_training_steps} \
 --class_word {class_word} \
 --no-test

## Pruning (12GB to 2GB)
We are working on having this happen automatically (TODO: PR's welcome)

In [ ]:
%%notify
directory_paths = !ls -d logs/*

# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
training_samples = !ls training_samples
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_" + class_word + ".ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{file_name}

In [ ]:
# Download your trained model file from `trained_models` and use in your favorite Stable Diffusion repo!

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 4 \
 --n_iter 4 \
 --seed 8971238632 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "{token} {class_word}, painting by greg rutkowski and alphonse mucha, masterpiece, trending on artstation"